In [ ]:
import os
import tempfile

import gymnasium
import minetester
import numpy as np

temp_dir = tempfile.mkdtemp(prefix='minetester_')
game_dir = os.path.join(os.environ["CONDA_PREFIX"], "share/minetest/games/devtest")
# install devtest game if needed
if not os.path.exists(game_dir):
    os.makedirs(os.path.dirname(game_dir), exist_ok=True)
    os.symlink(os.path.realpath(os.path.join(os.getcwd(), "../../games/devtest")), game_dir)

env = minetester.MinetestEnv(
    artifact_dir=os.path.join(temp_dir, "artifacts"),
    game_dir=game_dir,
)
video_dir = os.path.join(temp_dir, "video")
empty_action = {"KEYS": np.ndarray(0), "MOUSE": [0, 0]}
with gymnasium.wrappers.RecordVideo(env, video_dir) as env:
    env.reset()
    for i in range(1000):
        if i % 10 == 0:
            action = env.action_space.sample()
        else:
            action = empty_action
        observation, reward, done, truncated, info = env.step(action)
        if done: break


In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open(os.path.join(video_dir, os.listdir(video_dir)[-1]), "r+b").read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode("ascii")))